In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, cohen_kappa_score
from model import DRModel
import import_ipynb
from data_prepration import train_bin_loader, test_bin_loader
#from data_preparation import train_bin_loader, test_bin_loader

Total learnable parameters: 1396642
Data ready!


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DRModel(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [3]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_bin_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_bin_loader)}")

Epoch 1, Loss: 0.5579276007154713
Epoch 2, Loss: 0.36119470129842346
Epoch 3, Loss: 0.3068017493123594
Epoch 4, Loss: 0.25631206320679706
Epoch 5, Loss: 0.23443703482980313
Epoch 6, Loss: 0.21801871450051016
Epoch 7, Loss: 0.20257998484632242
Epoch 8, Loss: 0.195762997088225
Epoch 9, Loss: 0.19067174520181573
Epoch 10, Loss: 0.18335661454045254
Epoch 11, Loss: 0.19736800342798233
Epoch 12, Loss: 0.17449565069830936
Epoch 13, Loss: 0.17983838267948316
Epoch 14, Loss: 0.17557290196418762
Epoch 15, Loss: 0.14697820822829785
Epoch 16, Loss: 0.15022624186847522
Epoch 17, Loss: 0.14688335261915042
Epoch 18, Loss: 0.14212449445672656
Epoch 19, Loss: 0.14909582002007443
Epoch 20, Loss: 0.14334850531557333
Epoch 21, Loss: 0.14478452471287354
Epoch 22, Loss: 0.1374476415955502
Epoch 23, Loss: 0.13226548323164816
Epoch 24, Loss: 0.12692376647306525
Epoch 25, Loss: 0.12656124519265216
Epoch 26, Loss: 0.12523868647606476
Epoch 27, Loss: 0.12319763118158215
Epoch 28, Loss: 0.10936327602552331
Epoch 

In [4]:
# Evaluate
model.eval()
preds, trues = [], []
with torch.no_grad():
    for inputs, labels in test_bin_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.cpu().numpy())
        trues.extend(labels.numpy())
acc = accuracy_score(trues, preds)
kappa = cohen_kappa_score(trues, preds)
print(f"Binary Accuracy: {acc*100:.2f}%, Kappa: {kappa:.2f}")

Binary Accuracy: 90.45%, Kappa: 0.81


In [5]:
# Save model
torch.save(model.state_dict(), 'binary_model.pth')